# Modell evaluation
Dieses Notebook kann verwendet werden um nach dem Trainieren die Diagramme zur evaluierung zu erstellen. 

In [2]:
import pandas as pd
import plotly.express as px
WIDTH=800
HEIGHT=450

paths = {"size_n":r"C:\Users\malte.iwanicki\Documents\bachelor\BachelorInformatikAbschlussarbeit\src\runs\segment\size_n\results.csv",
         "size_m":r"C:\Users\malte.iwanicki\Documents\bachelor\BachelorInformatikAbschlussarbeit\src\runs\segment\size_m\results.csv"}

In [41]:
size_n = pd.read_csv(paths["size_n"])
size_n["Name"]="size_n"

size_m = pd.read_csv(paths["size_m"])
size_m["Name"]="size_m"
df = pd.concat([size_n,size_m])

In [ ]:
df.columns=[col.strip() for col in df.columns]

Index(['epoch', 'train/box_loss', 'train/seg_loss', 'train/cls_loss',
       'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)',
       'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)',
       'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)',
       'val/box_loss', 'val/seg_loss', 'val/cls_loss', 'val/dfl_loss',
       'lr/pg0', 'lr/pg1', 'lr/pg2', 'Name'],
      dtype='object')

In [43]:
def get_loss_df(df,loss):
    train = df[["Name","epoch",f"train/{loss}"]].copy()
    train["type"] = "train"
    train.columns=["Name", "epoch", "loss","type"]
    val = df[["Name","epoch",f"val/{loss}"]].copy()
    val["type"]= "val"
    val.columns=["Name", "epoch", "loss","type"]
    df = pd.concat([train,val],ignore_index=True)
    df.sort_values(["Name","epoch"],inplace=True, ignore_index=True)
    return {loss:df}

In [44]:
loss_types=["box_loss","seg_loss","cls_loss","dfl_loss"]
loss={}
_=[loss.update(get_loss_df(df, loss_type))for loss_type in loss_types]


In [45]:
def f1_score(precision, recall):
    try:
        return 2 * (precision * recall) / (precision + recall)
    except:
        return None
df["metrics/F1-Score"]=df.apply(lambda df: f1_score(df["metrics/precision(B)"],df["metrics/recall(B)"]), axis=1)

In [46]:
figs=[px.line(loss[loss_type],"epoch","loss",line_dash="type",color="Name",title=loss_type, width=WIDTH,height=HEIGHT) for loss_type in loss_types]

In [47]:
metrics = [col.split("/")[1] for col in df.columns if col.startswith("metrics")]
figs+= [px.line(df,"epoch",f"metrics/{metric}",title=metric, color="Name",width=WIDTH,height=HEIGHT) for metric in metrics]

In [48]:
_=[fig.show() for fig in figs]

[None, None, None, None, None, None, None, None, None, None, None, None, None]

# Time 

In [ ]:
import requests
import os
import time
from tqdm import tqdm
import plotly.express as px
import pandas as pd
image_folder = "datasets/UNIMIB2016/val/images"
upload_url = "http://127.0.0.1:8000/upload"


In [ ]:

def upload_images_and_measure_time(image_folder, upload_url):
    image_files = os.listdir(image_folder)
    warmed_up=False
    elapsed_times=[]

    for image_file in tqdm(image_files):
        image_path = os.path.join(image_folder, image_file)
        
        with open(image_path, "rb") as img:
            files = {"file": img}    
            if not warmed_up:
                requests.post(upload_url, files=files)
                warmed_up=True
                continue

            start_time = time.time()

            # Upload the image
            requests.post(upload_url, files=files)

            end_time = time.time()
            elapsed_time = end_time - start_time
            elapsed_times.append(elapsed_time)
    return elapsed_times
elapsed_times_m = upload_images_and_measure_time(image_folder, upload_url)


In [ ]:
medium = pd.read_csv(r"C:\Users\malte.iwanicki\Documents\bachelor\BachelorInformatikAbschlussarbeit\src\app\times_m.csv", ).max()
nano = pd.read_csv(r"C:\Users\malte.iwanicki\Documents\bachelor\BachelorInformatikAbschlussarbeit\src\app\times_n.csv", ).min()

In [ ]:
medium["gesamt"]=pd.DataFrame({"sekunden":elapsed_times_m}).mean().values[0]
nano["gesamt"]=pd.DataFrame({"sekunden":elapsed_times_n}).mean().values[0]

medium.name="medium"
nano.name="nano"

In [ ]:
get_other_times = lambda s: s["gesamt"]-s["Bilderkennung"]

In [ ]:
nano["Andere"]=get_other_times(nano)
medium["Andere"]=get_other_times(medium)

In [ ]:
df = pd.concat([nano.drop(["gesamt"]),medium.drop(["gesamt",])], keys=["Nano","Medium"]).reset_index(name="sekunden",)
df.columns=["Modell","Komponente","Sekunden"]
df

In [ ]:
px.bar(df,"Modell", "Sekunden",color="Komponente",)


In [ ]:
fig=px.pie(df,names="Komponente",values="Sekunden", facet_col="Modell")
fig.update_traces(textposition='inside')

In [ ]:
df["webserver"]=df["gesamt"]-df["image"]-df["db"]
df.drop(columns="gesamt", inplace=True)
df=df.reset_index(names="modell")

In [ ]:
times=pd.DataFrame({"Nano":elapsed_times_n,"Medium":elapsed_times_m})
fig = px.hist(times)
fig.update_layout( xaxis_title="Modell",
    yaxis_title="Sekunden")

In [ ]:
from foodscanner.image_recognition.image_recognition import ImageRecognition

MODEL_M = r"C:\Users\malte.iwanicki\Documents\bachelor\BachelorInformatikAbschlussarbeit\src\runs\segment\size_m\weights\best.pt"

MODEL_N = r"C:\Users\malte.iwanicki\Documents\bachelor\BachelorInformatikAbschlussarbeit\src\runs\segment\size_n\weights\best.pt"

def recognize_and_measure_time(model_path,image_folder):
    image_files = os.listdir(image_folder)
    elapsed_times = []
    ir = ImageRecognition(model_path)
    for image_file in tqdm(image_files):
        image_path = os.path.join(image_folder, image_file)
        start_time = time.time()

        # Upload the image
        ir.predict(image_path)

        end_time = time.time()
        elapsed_time = end_time - start_time
        elapsed_times.append(elapsed_time)
    return elapsed_times


#elapsed_times_m = recognize_and_measure_time(MODEL_M, image_folder)

elapsed_times_n = recognize_and_measure_time(MODEL_N, image_folder)


In [ ]:
x=pd.DataFrame({"Medium":elapsed_times_m[1:],"Nano":elapsed_times_n[1:]})

In [ ]:
fig =px.histogram(x,histnorm='percent', labels={"variable":"Modell"},nbins=1000)
fig.update_layout(xaxis_title="Sekunden", yaxis_title="Anteil in %",xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.2
    ))
fig.update_xaxes(range=[0.1, 1.6], fixedrange=True)

fig

In [ ]:
pd.DataFrame({"m":elapsed_times_m,"n":elapsed_times_n}).mean()